In [15]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout,GridspecLayout,interact, interact_manual,IntProgress,VBox,HBox,Button, ButtonStyle, Output,FileUpload
from IPython.display import HTML, display, clear_output, FileLink, FileLinks
import os
import subprocess
import lxml
import base64
from pymarc import marcxml,Record,Field

In [45]:
collection = marcxml.parse_xml_to_array("source_files/ap1.xml",strict=True)

In [18]:
"""General utility functions"""

# * * * * * * * * * * * * * * * * * *

def get_pymarc_field_value(tag,record):
    """default function for returning string of marc field value"""

    if record[tag]: return record[tag].format_field()
    else: return ""

def get_pymarc_subfield_value(tag,subfield,record):
    """default function for returning string of marc field value"""
    
    if record[tag] and record[tag][subfield]: return record[tag][subfield]
    else: return ""

def get_subfield_values(tag,subfield,record):
    """default function for returning a stringified list of subfield values"""

    values = []
    fields = record.get_fields(tag)

    if fields:
        for field in fields:

            for subfield in field.get_subfields(subfield):
                values.append(subfield.replace('\\',''))

    return str(values) if values else ""

In [49]:
def get_marc_metadata(record):
    result = {}
    result["titre"] = record['245']['a']
    for f in record.get_fields('020'):
        if f.get_subfields('q')[0] == '(Print-ISBN)':
            result["p-isbn"] = f.get_subfields('a')[0]
        elif f.get_subfields('q')[0] == '(E-ISBN)':
            result["e-isbn"] = f.get_subfields('a')[0]
    result["editeur"] = record['260']['b']
    result["date"] = record['260']['c']
    return result

In [50]:
df_result = pd.DataFrame(get_marc_metadata(record) for record in collection)

In [51]:
df_result

,titre,e-isbn,p-isbn,editeur,date
0,La problématique paternelle,9782749226453,9782865869121,"Érès,",2001
1,La résilience : le réalisme de l'espérance,9782749226576,9782865868223,"Érès,",2005
2,Perversions,9782749228518,9782749206639,"Érès,",2006
3,"Le jeune enfant, ses professionnels et la douleur",9782749227115,9782749207940,"Érès,",2007
4,Drogues et substitution,9782804173739,9782804150563,"De Boeck Supérieur,",2006
5,Lire Dolto aujourd'hui,9782749228037,9782749203096,"Érès,",2008
6,Psychanalyse et psychiatrie,9782749228891,9782865869626,"Érès,",2001
7,La jouissance au fil de l'enseignement de Lacan,9782749226255,9782749210599,"Érès,",2009
8,La preuve par la parole,9782749222462,9782749208459,"Érès,",2008
9,Psychologie du risque,9782804179274,9782804155438,"De Boeck Supérieur,",2007
